In [2]:
import numpy as np
import pandas as pd

# Исходные данные

In [6]:
n = 2 # количество свободных членов
m = 3 # количество базисных членов
b1=321
b2=356
b3=30

a11=2
a12=3
a13=32

a21=21
a22=4
a23=11

a31=13
a32=8
a33=2
c1 = -5
c2 = -2
c3 = -7

stroka1 = [b1,a11,a12,a13,1,0,0,None]
stroka2 = [b2,a21,a22,a23,0,1,0,None]
stroka3 = [b3,a31,a32,a33,0,0,1,None]
L = [0,c1,c2,c3,0,0,0,None]

Members = np.array([stroka1,stroka2,stroka3,L])
# Members = np.empty([n+2,m+5],dtype='float32')
B=[b1,b2,b3]
for i in range(len(B)):
    Members[i][0] = B[i]

print(Members)


[[321 2 3 32 1 0 0 None]
 [356 21 4 11 0 1 0 None]
 [30 13 8 2 0 0 1 None]
 [0 -5 -2 -7 0 0 0 None]]


# Создание симплексной таблицы

In [7]:
index = ["x4","x5","x6","L"]
columns = ["Свободные члены","x1","x2","x3","x4","x5","x6","Симплексные отношения"]
simplexTable = pd.DataFrame(Members,columns =columns,index = index)

In [8]:
# simplexTable.index([])
# simplexTable.loc('x1')
simplexTable

,Свободные члены,x1,x2,x3,x4,x5,x6,Симплексные отношения
x4,321,2,3,32,1,0,0,None
x5,356,21,4,11,0,1,0,None
x6,30,13,8,2,0,0,1,None
L,0,-5,-2,-7,0,0,0,None


# Реализация симплексного метода

In [9]:
flag = False
count = 0
while not flag:
    count+=1

    sli = simplexTable.loc["L","x1":"x3"]
    stolb = sli[sli==sli.min()].index[0] # Столбец с разрешающим элементом
    minCol = columns.index(stolb) # Колонка с потенциальным разрешающим элементом
    
    simplexTable["Симплексные отношения"] = simplexTable["Свободные члены"]/simplexTable[stolb] # Расчет симплексных онтношений

    print("############################")
    print("Итерация номер:",count)
    print("Исходная таблица")
    print(simplexTable)
    
    series = simplexTable["Симплексные отношения"] 
    minindex = simplexTable["Симплексные отношения"][:-1].min() 
    print(minindex)
    minindex = series[series == minindex].index[0] # нахождение строки с разрешающим элементом
    indexes = simplexTable.index 
    minI=index.index(indexes[indexes==minindex][0]) # нахождение индекса строки с разрешающим элементом

    indexes[indexes==minindex]
    index[minI] = stolb
    # simplexTable.reset_index()
    simplexTable.index = index # замена базисной переменной на новую разрешающую
    print("Замена базисной переменной на новую разрешающую")
    print(simplexTable)
    print("Колонка с разрешающим элементом", minCol)
    print("Строка с разрешающим элементом", minI)

    
    simplexTable.loc[stolb,:"x6"] = simplexTable.loc[stolb,:"x6"]/simplexTable[stolb][stolb] # приведение разрешающего элемента к базисному виду
    print("Приведение разрешающего элемента к базисному виду")
    print(simplexTable)
    # подстановка в ограничения разрешающего элемента и пересчет таблицы
    for i in range(len(index)):
        if i!=minI:
            simplexTable.iloc[i,:] = simplexTable.iloc[i,:] + simplexTable.iloc[minI,:]*(-simplexTable.iloc[i,minCol])
    print("Подстановка в ограничения разрешающего элемента и пересчет таблицы")   
    print(simplexTable)
    # Проверка на коэфициентов при переменных
    L = simplexTable.iloc[-1,1:-1]
    if len(L[L<0]) != 0:
        flag = False
    else: flag = True
simplexTable

############################
Итерация номер: 1
Исходная таблица
   Свободные члены  x1  x2  x3 x4 x5 x6 Симплексные отношения
x4             321   2   3  32  1  0  0               10.0312
x5             356  21   4  11  0  1  0               32.3636
x6              30  13   8   2  0  0  1                    15
L                0  -5  -2  -7  0  0  0                    -0
10.03125
Замена базисной переменной на новую разрешающую
   Свободные члены  x1  x2  x3 x4 x5 x6 Симплексные отношения
x3             321   2   3  32  1  0  0               10.0312
x5             356  21   4  11  0  1  0               32.3636
x6              30  13   8   2  0  0  1                    15
L                0  -5  -2  -7  0  0  0                    -0
Колонка с разрешающим элементом 3
Строка с разрешающим элементом 0
Приведение разрешающего элемента к базисному виду
   Свободные члены      x1       x2  x3       x4 x5 x6 Симплексные отношения
x3         10.0312  0.0625  0.09375   1  0.03125  0  0           

,Свободные члены,x1,x2,x3,x4,x5,x6,Симплексные отношения
x3,9.98301,0,0.0558252,1,0.0315534,0,-0.00485437,160.452
x5,229.978,0,-9.3568,0,-0.245146,1,-1.57767,-3.58425
x1,0.771845,1,0.606796,0,-0.00485437,0,0.0776699,0.771845
L,73.7403,0,1.42476,0,0.196602,0,0.354369,-11.8689


# В результате прибыль составляет 73,74 тысячи при производстве только 1,42 экземпляров оборудования x2